In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import metrics

In [5]:
with open('./pickle/Brownspot_lap0.pickle', 'rb') as handle:
    Brownspot_lap0 = pickle.load(handle)
    
with open('./pickle/Brownspot_lap1.pickle', 'rb') as handle:
    Brownspot_lap1 = pickle.load(handle)

with open('./pickle/Brownspot_lap2.pickle', 'rb') as handle:
    Brownspot_lap2 = pickle.load(handle)
    
with open('./pickle/Healthy_lap0.pickle', 'rb') as handle:
    Healthy_lap0 = pickle.load(handle)
    
with open('./pickle/Healthy_lap1.pickle', 'rb') as handle:
    Healthy_lap1 = pickle.load(handle)

with open('./pickle/Healthy_lap2.pickle', 'rb') as handle:
    Healthy_lap2 = pickle.load(handle)
    
with open('./pickle/sum_of_contoursArea_BrownSpot.pickle', 'rb') as handle:
    sum_of_contoursArea_BrownSpot = pickle.load(handle)

with open('./pickle/sum_of_contoursArea_Healthy.pickle', 'rb') as handle:
    sum_of_contoursArea_Healthy = pickle.load(handle)
    
with open('./pickle/secondLargestArea_BrownSpot.pickle', 'rb') as handle:
    secondLargestArea_BrownSpot = pickle.load(handle)

with open('./pickle/secondLargestArea_Healthy.pickle', 'rb') as handle:
    secondLargestArea_Healthy = pickle.load(handle)

In [6]:
data = []

data_level = []

for i in range(0, len(Brownspot_lap0)):
    data.append([Brownspot_lap0[i], Brownspot_lap1[i], Brownspot_lap2[i], sum_of_contoursArea_BrownSpot[i], secondLargestArea_BrownSpot[i]])
    data_level.append(0)
    data.append([Healthy_lap0[i], Healthy_lap1[i], Healthy_lap2[i], sum_of_contoursArea_Healthy[i], secondLargestArea_Healthy[i]])
    data_level.append(1)
    
print(len(data_level))
print(len(data))

1046
1046


In [7]:
ndata = np.array(data)
ndata_level = np.array(data_level)

In [8]:
df=pd.DataFrame({
    'laplacian red':ndata[:,0],
    'laplacian green':ndata[:,1],
    'laplacian blue':ndata[:,2],
    'sum of contours area':ndata[:,3],
    'second largest area':ndata[:, 4],
    'class':ndata_level[:]
})

In [9]:
X=df[['laplacian red', 'laplacian green', 'laplacian blue', 'sum of contours area', 'second largest area']]  # Features
y=df['class']  # Labels

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
X_test.shape

(210, 5)

In [34]:
params = {'n_estimators':[30, 50, 70], 'criterion':['gini', 'entropy'], 'max_depth':range(1, 5)}

In [39]:
model = RandomForestClassifier()
grid_search = GridSearchCV(model,param_grid=params,cv=5,scoring='accuracy')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [30, 50, 70], 'criterion': ['gini', 'entropy'], 'max_depth': range(1, 5)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [40]:
print(grid_search.best_params_)

{'criterion': 'entropy', 'max_depth': 4, 'n_estimators': 70}


In [41]:
grid_search.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=70, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [261]:
model = RandomForestClassifier(n_estimators=70, criterion='entropy', max_depth=4)

In [262]:
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=70, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [263]:
y_pred=model.predict(X_test)

In [264]:
print(confusion_matrix(y_test, y_pred))

[[91 23]
 [ 5 91]]


In [265]:
print(accuracy_score(y_test, y_pred))

0.8666666666666667


In [266]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.80      0.87       114
           1       0.80      0.95      0.87        96

   micro avg       0.87      0.87      0.87       210
   macro avg       0.87      0.87      0.87       210
weighted avg       0.88      0.87      0.87       210



In [157]:
from sklearn.externals import joblib

joblib.dump(model,"random_forest_model.pkl")

C:\Users\Amit\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\Amit\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\Amit\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\Amit\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\Amit\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_hand

['random_forest_model.pkl']